In [1]:
import pandas as pd 
import numpy as np
from scipy.special import comb
import math
from neal import SimulatedAnnealingSampler
from pyqubo import Array, Constraint, Placeholder, solve_qubo
import itertools
import random
import matplotlib.pyplot as plt

In [2]:
#!pip3 install dwave-system

In [3]:
from dwave.system.composites import AutoEmbeddingComposite
from dwave.system.samplers import DWaveSampler
from pyqubo import Array, Constraint, Placeholder, solve_qubo

In [4]:
DWAVE_TOKEN="TOKY-1319d5c52b9aa35f34b40feba0cea58a4f5d3c09" #"replace this string with actual token"

### 解析用データ作成

（3年間骨肉腫に罹患しない）事象と各特徴量とのP値を計算する。
- Osteogenic sarcoma data:骨肉腫
- LI:リンパ球浸出
- AOP:類骨病理学

In [5]:
import pandas as pd
import numpy as np

In [6]:
ost30 = pd.read_csv("../../input/ost30.csv", sep=',', index_col=0)
ost30.shape

(30, 4)

In [7]:
ost30.head()

,Y,LI,SEX,AOP
0,1,1,1,1
1,0,1,1,0
2,1,0,0,0
3,1,0,0,0
4,1,0,1,0


In [8]:
t_list = []
t_list.append(sum(ost30['Y']))
t_list.append(np.dot(ost30['Y'], ost30['LI']))
t_list.append(np.dot(ost30['Y'], ost30['SEX']))
t_list.append(np.dot(ost30['Y'], ost30['AOP']))

In [9]:
t_list#Y, LI, SEX, AOP

[18, 11, 5, 7]

In [10]:
columns = list(ost30.columns)
num_samples = ost30.shape[0]

### annealing_time = 20, num_reads=100, for t1 in range(0, ost16.shape[0]+1)

In [11]:
dw_sampler = DWaveSampler(
    endpoint="https://cloud.dwavesys.com/sapi",
    solver = 'DW_2000Q_6',
    token = DWAVE_TOKEN
)

In [12]:
qa_sampler = AutoEmbeddingComposite(dw_sampler)

In [13]:
def find_valid_y(df, num_reads):
    valid_y_list = {}
    valid_y_num = {}
    for t1 in range(0, df.shape[0]+1):
        y = Array.create('y', shape=df.shape[0], vartype='BINARY')

        #QUBO式で定式化
        H = (sum(y) - t_list[0])**2
        H += (np.dot(df[columns[1]], y) - t1)**2
        for j in range(2, len(columns)):
            H_plus = (np.dot(df[columns[j]], y) - t_list[j])**2
            H += H_plus

        qb = H.compile().to_qubo()
        res = qa_sampler.sample_qubo(qb[0], num_reads=100)

        valid_y_list[t1] = []
        valid_y_num[t1] = 0
        for y_info in list(res.record):
            if sum(y_info[0]) == t_list[0]:#int同士の比較
                y = pd.Series(y_info[0])
                if np.dot(df[columns[1]], y) == t1:#int同士の比較
                    for j in range(2, len(columns)):
                        if np.dot(df[columns[j]], y) != t_list[j]:#int同士の比較
                            break
                    else:
                        if all(list(y_info[0]) != p for p in valid_y_list[t1]):
                            valid_y_num[t1] += 1
                            valid_y_list[t1].append(list(y_info[0]))
                            print('perfect')
    return valid_y_list, valid_y_num

In [14]:
import time

# 処理前の時刻
t1 = time.time() 

In [15]:
%%timeit

valid_y_list, valid_y_num = find_valid_y(ost30, 100)

2min 16s ± 5.01 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
# 処理後の時刻
t2 = time.time()

# 経過時間を表示
elapsed_time = t2-t1
print(f"経過時間：{elapsed_time}秒")

経過時間：1096.3004367351532秒


In [17]:
valid_y_list

NameError: name 'valid_y_list' is not defined

In [ ]:
valid_y_num

In [ ]:
valid_y_num[9]/sum(valid_y_num.values())